## Data import and setting workspace

In [ ]:
import arcpy
from arcpy import env
import os
from arcpy.sa import *

In [ ]:
# Set local variable
arcpy.env.workspace = r"C:\EsriTraining\wetland_analysis"
arcpy.env.overwriteOutput = True

# Create FileGDB
arcpy.management.CreateFileGDB(
    out_folder_path=r"C:\EsriTraining\wetland_analysis",
    out_name="project644",
    out_version="CURRENT"
)

In [ ]:
# Set output geodatabase
out_gdb = r"C:\EsriTraining\wetland_analysis\project644.gdb"

# Get list of shapefiles in workspace
featureclasses = arcpy.ListFeatureClasses()
#print(featureclasses)

# Loop through each shapefile and project to out_gdb
for shapefile in featureclasses:
    # Set input shapefile path
    in_shapefile = os.path.join(arcpy.env.workspace, shapefile)
    
    # Set output feature class name
    out_feature_class = os.path.join(out_gdb, f"{os.path.splitext(shapefile)[0]}_prj")
    
    # Project shapefile to out_gdb
    arcpy.management.Project(
        in_dataset=in_shapefile,
        out_dataset=out_feature_class,
        out_coor_system='PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]',
        transform_method=None,
        in_coor_system='PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]',
        preserve_shape="NO_PRESERVE_SHAPE",
        max_deviation=None,
        vertical="NO_VERTICAL"
    )
    
# Create bounding box from polygon
env.workspace = r"C:\EsriTraining\wetland_analysis\project644.gdb"

# Create variables for the input and output feature classes
inFeatures = "aoi_boundary_prj"
outFeatureClass = "bbox"

# Use MinimumBoundingGeometry function to get a bbox area 
arcpy.MinimumBoundingGeometry_management(inFeatures, outFeatureClass, 
                                         "ENVELOPE", "NONE")

In [ ]:
## Project all rasters files 
input_folder = r"C:\EsriTraining\wetland_analysis\naip"
output_folder = r"C:\EsriTraining\wetland_analysis\projected"

# Set output coordinate system
out_coor_system = 'PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]'

# Set other parameters
resampling_type = "NEAREST"
cell_size = "0.599999999999999 0.600000000000029"
geographic_transform = "WGS_1984_(ITRF00)_To_NAD_1983"
registration_point = None
in_coor_system = 'PROJCS["NAD_1983_UTM_Zone_17N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-81.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'
vertical = "NO_VERTICAL"

# Iterate through all files in input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.tif'):  # Only process TIFF files
        # Set input and output paths
        in_raster = os.path.join(input_folder, filename)
        out_raster = os.path.join(output_folder, filename[20:-4]) + ".tif"
        
        # Project raster
        arcpy.management.ProjectRaster(in_raster, out_raster, out_coor_system, resampling_type, cell_size, geographic_transform, registration_point, in_coor_system, vertical)

## Clip & NDVI analysis 

In [ ]:
# set workspace
arcpy.env.workspace = r"C:\EsriTraining\wetland_analysis\projected"
workspace = arcpy.env.workspace

# Set output folder
output_folder = os.path.join(workspace, "clip")
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Creating bounding box for aoi (minimum boundary geometry)
boundary_layer = r"C:\EsriTraining\wetland_analysis\project644.gdb\aoi_boundary_prj"
boundary_desc = arcpy.Describe(boundary_layer)
analysis_extent = boundary_desc.extent

# loop through raster files and clip
raster_list = arcpy.ListRasters("*", "TIF")
for raster in raster_list:
    # set input and output paths
    in_raster = os.path.join(workspace, raster)
    out_raster = os.path.join(output_folder, "clipped_" + raster[:-4] + ".tif")

    # clip raster
    out_clip = arcpy.sa.ExtractByMask(in_raster, boundary_layer)
    out_clip = arcpy.sa.ExtractByRectangle(out_clip, analysis_extent)
    out_clip.save(out_raster)
    
    print(f"{out_raster} clipped successfully!")

In [ ]:
output_folder = r"C:\EsriTraining\wetland_analysis\ndvi"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

arcpy.env.workspace = r"C:\EsriTraining\wetland_analysis\projected\clip"
workspace = arcpy.env.workspace

raster_list = arcpy.ListRasters("*", "TIF")
for raster in raster_list:
    in_raster = os.path.join(arcpy.env.workspace, raster)
    out_raster = os.path.join(output_folder, "ndvi_" + raster[:-4] + ".tif")
    
    red_band = Raster(in_raster + "/Band_1")
    nir_band = Raster(in_raster + "/Band_4")
    ndvi = (nir_band - red_band) / (nir_band + red_band)
        
    # Save the NDVI output to the output folder
    ndvi.save(out_raster)
    
    print(f"{out_raster} calculated successfully!")

## Animate rasters

In [ ]:
import os
import imageio
import imageio.v2 as imageio
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.pyplot as mplot
import matplotlib.image as mpimg
import matplotlib.colors as colors
from matplotlib.colors import ListedColormap
import rasterio
from rasterio.plot import show, show_hist

In [ ]:
inDir = r'C:\EsriTraining\wetland_analysis\ndvi'
os.chdir(inDir)
outDir = os.path.normpath(os.path.split(inDir)[0]+os.sep+'animation')
if not os.path.exists(outDir):
    os.makedirs(outDir)

In [ ]:
fileList = [file for file in os.listdir() if file.endswith('.tif')]
for i, f in enumerate(fileList):
    lcpri_file = rasterio.open(f)
    lcpri = lcpri_file.read(1)
    
    cmap = "YlGn"
    cmap_reversed = matplotlib.cm.get_cmap('YlGn_r')
    plt.figure(figsize=(20,15))
    plt.imshow(lcpri, cmap=cmap, vmax=0.5, vmin=-0.5)
    parts = f.split("_")
    fileName = parts[-1][:-4]
    print('Processing: {}'.format(fileName))
    plt.title('NDVI' + ' ' + fileName, fontsize=25)
    cur_axes = plt.gca()
    cur_axes.axes.get_xaxis().set_visible(False)
    cur_axes.axes.get_yaxis().set_visible(False)
    cb = plt.colorbar()
    cb.set_label('NDVI Time Series', size=20)
    cb.ax.tick_params(labelsize=18)
    plt.tight_layout()
    plt.savefig(os.path.join(outDir, '{}_NDVI_{}.png'.format(fileName, i)), dpi=150)
    plt.close()

In [ ]:
def make_gif(input_folder, save_filepath):
    episode_frames = []
    time_per_step = 0.30
    for root, _, files in os.walk(input_folder):
        file_paths = [os.path.join(root, file) for file in files]
        file_paths = sorted(file_paths, key = lambda x:os.path.getmtime(x))
        episode_frames = [imageio.imread(file_path)
                            for file_path in file_paths if file_path.endswith('.png')]
        episode_frames = np.array(episode_frames)
        imageio.mimsave(save_filepath,episode_frames,duration=time_per_step)
make_gif(outDir, os.path.join(outDir, "NDVI_Animation.gif"))
print("Create animation !!!")

In [ ]:
from IPython.display import Image, display
with open(r'C:\EsriTraining\wetland_analysis\animation\NDVI_Animation.gif', 'rb') as f:
    display(Image(data=f.read(), format = 'png'))